In [1]:
#from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plotly
#import plotly.offline as py
#import plotly.graph_objs as go
#py.init_notebook_mode(connected=True)

# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords

#from sklearn.manifold import TSNE

Using TensorFlow backend.


In [2]:
df=pd.read_csv("preprocessing_done.csv")

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
df.answer1=[" ".join(i.split()[:100])  if len(i.split())>100 else  i for i in df.answer1]

In [6]:
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
glove_file = datapath('glove.6B.100d.txt')
tmp_file = get_tmpfile("test_word2vec.txt")
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec(glove_file, tmp_file)
model = KeyedVectors.load_word2vec_format(tmp_file)

In [7]:
modelx=model

In [8]:
df.answer1=df.answer1.str.replace('dcu','')
df.question1=df.question1.str.replace('dcu','')

In [9]:
def matrix_generation(item):
    target_vocab=item.split()
    weights_matrix = np.zeros((100, 100))
    for i, word in enumerate(target_vocab):
        try: 
            weights_matrix[i] = modelx[word]
        except KeyError:
            weights_matrix[i] = np.random.normal(0.0, 1.0, (100,))
    return weights_matrix

In [10]:
question_matrices=df.question1.apply(matrix_generation)
answer_matrices=df.answer1.apply(matrix_generation)

In [11]:
def transformation(weight_matrices):
    x=[i for i in weight_matrices ]
    y=np.dstack(x)
    y=np.rollaxis(y,-1)
    y=y.reshape(518,1, 100, 100)
    return y

In [12]:
question_matrices=transformation(question_matrices)
answer_matrices=transformation(answer_matrices)

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Dropout,Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, GlobalMaxPooling1D

seq_length = 100

model = Sequential()
model.add(Conv1D(64, 3, activation='relu', input_shape=(seq_length, 100)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))

In [14]:
def vector_generator(weight_matrix):   
    feature_vector = model.predict(weight_matrix)
    return feature_vector

In [15]:
df=df.assign(answer_vectors =[vector_generator(i) for i in answer_matrices ])

In [16]:
df=df.assign(question_vectors =[vector_generator(i) for i in question_matrices ])

In [18]:
from sklearn.metrics.pairwise import cosine_similarity
df=df.assign(similarity=[cosine_similarity(df.question_vectors[i], df.answer_vectors[i]) for i in range(0,518)])

In [190]:
df.head(2)

,Question,Answer,question1,answer1,answer_vectors,question_vectors,similarity
0,What Is A Mortgage?,A mortgage is a loan that finances the purchas...,mortgage,mortgage loan finance purchase home property m...,"[[0.013478862, 0.0, 0.1825618, 0.08606828, 0.1...","[[0.0, 0.0, 0.0005435275, 0.0006598317, 0.0, 4...",[[0.59051615]]
1,"When Purchasing A Home In Ottawa, What Is The ...","Generally, a minimum down payment of 5% is req...",purchase home ottawa minimum payment i ll pay,generally minimum payment require apply mortg...,"[[0.0, 0.0, 0.12924471, 0.0498013, 0.069889374...","[[0.0, 0.0, 0.020613687, 0.011827095, 0.002554...",[[0.8409533]]


In [209]:
#inputx="reverse mortgage" #question
#inputx="standard variable rate mortgage" #question
inputx="mortgage" #answer
#inputx="provide personal data application secure" #answer


In [210]:
inputx=matrix_generation(inputx)
inputx=inputx.reshape(1,100,100)
inputx=vector_generator(inputx)
x=np.array([cosine_similarity(inputx,df.question_vectors[i])+0.0*df.similarity[i]  for i in range(0,518)]).argsort(axis=0)[-20:]

In [211]:
for i in np.fliplr([x])[0]:
    print df.Question[i[0][0]]

What Is A Mortgage?
What is a mortgage?
What is a Mortgage?
What is a mortgage?
What is FHA?
Who are your Lenders?
What is my down payment?
What is a discount mortgage?
What is a commercial mortgage?
What is Freddie Mac?
What is an amortized mortgage?
What is HUD?
What is a repayment mortgage?
How do I re-mortgage?
What is the term of a loan?
What is an offset mortgage?
What is an Offset Mortgage?
What is a Conventional Mortgage?
What is a Conventional Mortgage?
Is there a fee to submit my application online?


In [202]:
for i in np.fliplr([x])[0]:
    print df.Answer[i[0][0]]

Just like a fixed interest rate mortgage, a standard variable interest rate can be applied to any type of mortgage. The rate can fluctuate at the lender's discretion and will generally change as a reaction to economic conditions and the Bank of England's base rate. That means your monthly payments can go up or down.\n\nA standard variable interest rate will usually be higher than the typical offerings on the market when you start searching for mortgages. However, some mortgage lenders offer 'capped' standard variable rates. These ensure that the interest rate will not increase by more than a maximum increment, will not rise above more than a set level, or will only change once during a certain period time.\n\nEach bank usually has one standard variable rate which is universal to all of its customers. However, some lenders will offer discounts to the standard variable rate for some customers, typically those within the early years of the mortgage term when the affordability levels are t